## Install Dependencies

In [101]:
# !pip install tensorflow
# !pip install gym
# !pip install keras
# !pip install keras-rl2

    opencv-python (>=3.) ; extra == 'all'
                  ~~~~^


## Test Random Environment with OpenAI Gym

In [24]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [26]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take: Down, Stay, Up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3, 3)
        # Set shower length
        self.shower_length = 60
    
    def step(self, action):
        # Apply action: 0, 1, 2
        self.state += action - 1
        # Reduce shower length by 1 second
        self.shower_length -= 1

        # Calculate Reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False

        # Apply tempurature noise
        self.state += random.randint(-1, 1)

        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info
    
    def render(self): # For Visualization, not used/implemented here
        pass
        
    def reset(self):
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        return self.state

In [28]:
env = ShowerEnv()

C:\Users\zemia\anaconda3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [84]:
env.action_space.sample()

2

In [42]:
env.observation_space.sample()

array([88.02127], dtype=float32)

In [5]:
# env = gym.make('CartPole-v0')
# states = env.observation_space.shape[0]
# actions = env.action_space.n

In [44]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        # env.render()
        # action = random.choice([0,1])
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-30
Episode:2 Score:-28
Episode:3 Score:-54
Episode:4 Score:-60
Episode:5 Score:-32
Episode:6 Score:-56
Episode:7 Score:-8
Episode:8 Score:-16
Episode:9 Score:-20
Episode:10 Score:-54


## Create a Deep Learning Model with Keras

In [86]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [88]:
states = env.observation_space.shape
actions = env.action_space.n

In [90]:
actions

3

In [92]:
def build_model(states, actions):
    model = Sequential()
    # model.add(Flatten(input_shape = (1, states)))
    model.add(Dense(24, activation = 'relu', input_shape = states))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [113]:
del model

In [115]:
model = build_model(states, actions)

In [117]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 24)                48        
                                                                 
 dense_8 (Dense)             (None, 24)                600       
                                                                 
 dense_9 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


## Build Agent with Keras-RL

In [120]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [122]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy, 
                  nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2)
    return dqn

In [124]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 5:29 - reward: 1.0000

C:\Users\zemia\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\zemia\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 25s 3ms/step - reward: -0.6520
166 episodes - episode_reward: -39.205 [-60.000, 32.000] - loss: 2.181 - mae: 12.289 - mean_q: -16.676

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 25s 3ms/step - reward: -0.5506
167 episodes - episode_reward: -33.042 [-60.000, 22.000] - loss: 3.089 - mae: 13.158 - mean_q: -18.958

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: -0.5024
167 episodes - episode_reward: -30.084 [-60.000, 18.000] - loss: 2.753 - mae: 12.062 - mean_q: -17.342

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: -0.3842
166 episodes - episode_reward: -22.964 [-60.000, 20.000] - loss: 2.168 - mae: 10.416 - mean_q: -14.865

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: -0.2132
done, took 128.455 seconds


In [126]:
scores = dqn.test(env, nb_episodes = 100, visualize = False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 58.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 58.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 58.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 58.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 58.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 58.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 58.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 58.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 60.000, st